In [1]:
import numpy as np
import pandas as pd
from mlcbakery import bakery_client
import pathlib
client = bakery_client.Client("http://localhost:8000", token="authtoken")

def add_rand_column_to_dataset(path_to_csv, column_name):
    df = pd.read_csv(path_to_csv)
    df[column_name] = np.random.randint(0, 100)
    df.to_csv(path_to_csv, index=False)


In [2]:
client.find_or_create_by_collection_name("test23")

BakeryCollection(id=3995, name='test23', description='', storage_info=None, storage_provider=None)

In [2]:
attributed_to = "jon@jetty.io"
# prepares the dataset folder for the bakery by creating a .manifest.json file based on the folder structure
client.prepare_dataset("./datasets/A1_000", params={
    "properties": {
        "type": "dataset",
        "name": "A1_001",
        "collection_name": "test23",
        "origin": "jetty.io",
        "metadata_version": "1.0.0",
    },
    "parents": [
        {
            "generated_by": None,
            "attributed_to": attributed_to
        }
    ],
    "assets": {
        "long_description": "README.md",
        "metadata": "metadata.json"
    }    
})

2025-05-12 03:54:15,844 - mlcbakery.bakery_client - INFO - Created .manifest.json in './datasets/A1_000'


{'properties': {'type': 'dataset',
  'name': 'A1_001',
  'collection_name': 'test23',
  'origin': 'jetty.io',
  'metadata_version': '1.0.0'},
 'parents': [{'generated_by': None, 'attributed_to': 'jon@jetty.io'}],
 'assets': {'long_description': 'README.md', 'metadata': 'metadata.json'}}

In [3]:
ds = client.save_to_bakery("./datasets/A1_000", upload_data=True)

2025-05-12 03:54:19,085 - absl - WARNING - WARNING: The JSON-LD `@context` is not standard. Refer to the official @context (e.g., from the example datasets in https://github.com/mlcommons/croissant/tree/main/datasets/1.0). The different keys are: {'fileProperty', 'conformsTo', 'rai', 'includes', 'references', 'transform', 'citeAs', 'isLiveDataset', 'separator', '@vocab', 'format', 'data', '@language', 'key', 'dct', 'fileSet', 'cr', 'examples', 'path', 'parentField', 'subField', 'regex', 'jsonPath', 'md5', 'repeated', 'fileObject', 'replace'}
2025-05-12 03:54:19,121 - mlcbakery.bakery_client - INFO - Created tar.gz of data folder at /tmp/tmpknwh62kw.tar.gz
2025-05-12 03:54:19,122 - mlcbakery.bakery_client - INFO - Pushing dataset 'test23/A1_001' to Bakery API
2025-05-12 03:54:19,313 - absl - WARNING - WARNING: The JSON-LD `@context` is not standard. Refer to the official @context (e.g., from the example datasets in https://github.com/mlcommons/croissant/tree/main/datasets/1.0). The diff

In [9]:
import json
# load json:
with open("../../bakerydev-bakery-storage-sa.json", "r") as f:
    storage_info = json.load(f)
client.update_collection_storage_info("test23", storage_info=storage_info, storage_provider="gcp")
get_storage_info = client.get_collection_storage_info("test23")

In [8]:
# ds = client.save_to_bakery("./datasets/A1_000", upload_data=True)
# ds

In [6]:
client.duplicate_dataset("datasets/A1_000", "datasets/B1_005", params={
    "properties": {
        "name": "B1_004"
    },
}, attributed_to="jon@jetty.io")
# add_rand_column_to_dataset("datasets/B1_000/data/data.csv", "B1")
ds = client.save_to_bakery("./datasets/B1_005", upload_data=True)

2025-05-12 03:55:55,332 - mlcbakery.bakery_client - INFO - Copied dataset from 'datasets/A1_000' to 'datasets/B1_005'
2025-05-12 03:55:55,333 - mlcbakery.bakery_client - INFO - Updated .manifest.json in 'datasets/B1_005'
2025-05-12 03:55:55,336 - absl - WARNING - WARNING: The JSON-LD `@context` is not standard. Refer to the official @context (e.g., from the example datasets in https://github.com/mlcommons/croissant/tree/main/datasets/1.0). The different keys are: {'fileProperty', 'conformsTo', 'rai', 'includes', 'references', 'transform', 'citeAs', 'isLiveDataset', 'separator', '@vocab', 'format', 'data', '@language', 'key', 'dct', 'fileSet', 'cr', 'examples', 'path', 'parentField', 'subField', 'regex', 'jsonPath', 'md5', 'repeated', 'fileObject', 'replace'}
2025-05-12 03:55:55,346 - mlcbakery.bakery_client - INFO - Created tar.gz of data folder at /tmp/tmp6n1liyqy.tar.gz
2025-05-12 03:55:55,347 - mlcbakery.bakery_client - INFO - Pushing dataset 'test23/B1_004' to Bakery API
2025-05-12

In [ ]:
# client.push_dataset("notebooks/datasets/A1_000")

# client.duplicate_dataset("notebooks/datasets/A1_000", "notebooks/datasets/B1_000", params={
#     "properties": {
#         "name": "B1_000"
#     },
# }, attributed_to=attributed_to)
# add_rand_column_to_dataset("notebooks/datasets/B1_000/data.csv", "B1")
# client.push_dataset("notebooks/datasets/B1_000")

In [1]:
# take dataset A_### 
# generate some metadata
# save the metadata + dataset as A1_###

# grab a dataset A1_###
# change the dataset + metadata
# save the metadata + dataset as B1_###

# grab a dataset A1_###
# change the dataset + metadata
# save the metadata + dataset as C1_###


# grab a dataset B1_###
# change the dataset + metadata
# save the metadata + dataset as D1_###




2

In [11]:
from mlcbakery import croissant_validation
result = croissant_validation.validate_json("../test.json")
croissant_validation.validate_croissant(result.valid_json_data)

ValidationResult(passed=True, message='The file is valid JSON.', details=None, valid_json_data={'@context': {'@language': 'en', '@vocab': 'https://schema.org/', 'arrayShape': 'cr:arrayShape', 'citeAs': 'cr:citeAs', 'column': 'cr:column', 'conformsTo': 'dct:conformsTo', 'cr': 'http://mlcommons.org/croissant/', 'data': {'@id': 'cr:data', '@type': '@json'}, 'dataBiases': 'cr:dataBiases', 'dataCollection': 'cr:dataCollection', 'dataType': {'@id': 'cr:dataType', '@type': '@vocab'}, 'dct': 'http://purl.org/dc/terms/', 'extract': 'cr:extract', 'field': 'cr:field', 'fileProperty': 'cr:fileProperty', 'fileObject': 'cr:fileObject', 'fileSet': 'cr:fileSet', 'format': 'cr:format', 'includes': 'cr:includes', 'isArray': 'cr:isArray', 'isLiveDataset': 'cr:isLiveDataset', 'jsonPath': 'cr:jsonPath', 'key': 'cr:key', 'md5': 'cr:md5', 'parentField': 'cr:parentField', 'path': 'cr:path', 'personalSensitiveInformation': 'cr:personalSensitiveInformation', 'recordSet': 'cr:recordSet', 'references': 'cr:refere